In [23]:
import cv2
import numpy as np
import os
from ultralytics import YOLO

model = YOLO("best.pt")
image_folder = "Task"
image_files = sorted(f for f in os.listdir(image_folder) if f.endswith('.jpg'))
image_pairs = [(image_files[i], image_files[i + 1]) for i in range(0, len(image_files) - 1, 2)]

unique_objects = []

def detect_objects(image_path):
    image = cv2.imread(image_path)
    if image is None:
        return [], None
    results = model(image)
    detections = results[0].boxes.data.cpu().numpy()
    return [(x1, y1, x2, y2, conf, int(class_id)) for x1, y1, x2, y2, conf, class_id in detections if int(class_id) == 1], image
   

def is_duplicate(new_object, existing_objects, threshold=0.5):
    x1, y1, x2, y2, _, _ = new_object
    for obj in existing_objects:
        ox1, oy1, ox2, oy2, _, _ = obj
        intersection_x1 = max(x1, ox1)
        intersection_y1 = max(y1, oy1)
        intersection_x2 = min(x2, ox2)
        intersection_y2 = min(y2, oy2)
        intersection_area = max(0, intersection_x2 - intersection_x1) * max(0, intersection_y2 - intersection_y1)
        area_new = (x2 - x1) * (y2 - y1)
        area_existing = (ox2 - ox1) * (oy2 - oy1)
        iou = intersection_area / (area_new + area_existing - intersection_area)
        if iou > threshold:
            return True
    return False

def draw_annotations(image, objects, label):
    colors = [(255, 0, 0), (255, 165, 0), (0, 255, 0)]
    for x1, y1, x2, y2, conf, class_id in objects:
        color = colors[class_id % len(colors)]
        cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
        cv2.putText(image, f"{class_id} {conf:.2f}", (int(x1), int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    cv2.putText(image, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

for front_image, back_image in image_pairs:
    front_objects, front_img = detect_objects(os.path.join(image_folder, front_image))
    back_objects, back_img = detect_objects(os.path.join(image_folder, back_image))

    
    unique_back_objects = [obj for obj in back_objects if not is_duplicate(obj, front_objects)]
    
    for obj in front_objects + unique_back_objects:
        if not is_duplicate(obj, unique_objects):
            unique_objects.append(obj)
    
    draw_annotations(front_img, front_objects, "Front")
    draw_annotations(back_img, unique_back_objects, "Back")
    
    combined_image = np.hstack((front_img, back_img))
    cv2.imshow(f"Comparison: {front_image} & {back_image}", combined_image)
    cv2.waitKey(0)

cv2.destroyAllWindows()
print(f"Total Unique Objects Counted: {len(unique_objects)}")


0: 480x640 4 1s, 1 3, 342.8ms
Speed: 0.0ms preprocess, 342.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 1s, 1 3, 261.9ms
Speed: 4.5ms preprocess, 261.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 2, 1 3, 335.9ms
Speed: 0.0ms preprocess, 335.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 3, 287.7ms
Speed: 3.0ms preprocess, 287.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 8 1s, 3 3s, 312.5ms
Speed: 2.9ms preprocess, 312.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 7 1s, 3 3s, 268.4ms
Speed: 4.0ms preprocess, 268.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 8 1s, 2 3s, 320.7ms
Speed: 8.8ms preprocess, 320.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 8 1s, 2 3s, 263.0ms
Speed: 3.8ms preprocess, 263.0ms inference, 1.3ms postprocess per ima